<a name="2"></a>
## 问题描述

Suppose you are starting a company that grows and sells wild mushrooms. 
- Since not all mushrooms are edible, you'd like to be able to tell whether a given mushroom is edible or poisonous based on it's physical attributes
- You have some existing data that you can use for this task. 

Can you use the data to help you identify which mushrooms can be sold safely? 

Note: The dataset used is for illustrative purposes only. It is not meant to be a guide on identifying edible mushrooms.



<a name="3"></a>
## 数据集

You will start by loading the dataset for this task. The dataset you have collected is as follows:

|                                                     | Cap Color | Stalk Shape | Solitary | Edible |
|:---------------------------------------------------:|:---------:|:-----------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |   Brown   |   Tapering  |    Yes   |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    Yes   |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |    Red    |  Enlarging  |    No    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    Yes   |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |    Red    |   Tapering  |    No    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |   Brown   |  Enlarging  |    No    |    0   |


-  You have 10 examples of mushrooms. For each example, you have
    - Three features
        - Cap Color (`Brown` or `Red`),
        - Stalk Shape (`Tapering (as in \/)` or `Enlarging (as in /\)`), and
        - Solitary (`Yes` or `No`)
    - Label
        - Edible (`1` indicating yes or `0` indicating poisonous)

<a name="3.1"></a>
### 3.1 One hot encoded dataset
For ease of implementation, we have one-hot encoded the features (turned them into 0 or 1 valued features)

|                                                    | Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:--------------------------------------------------:|:---------:|:--------------------:|:--------:|:------:|
| <img src="images/0.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/1.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/2.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/3.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |
| <img src="images/4.png" alt="drawing" width="50"/> |     1     |           1          |     1    |    1   |
| <img src="images/5.png" alt="drawing" width="50"/> |     0     |           1          |     1    |    0   |
| <img src="images/6.png" alt="drawing" width="50"/> |     0     |           0          |     0    |    0   |
| <img src="images/7.png" alt="drawing" width="50"/> |     1     |           0          |     1    |    1   |
| <img src="images/8.png" alt="drawing" width="50"/> |     0     |           1          |     0    |    1   |
| <img src="images/9.png" alt="drawing" width="50"/> |     1     |           0          |     0    |    0   |



In [34]:
import numpy as np
import matplotlib.pyplot as plt


In [35]:
# Data set
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

### Check the dimension of the dataset

In [36]:
X_train.shape, y_train.shape

((10, 3), (10,))

### Basic Function
entropy
$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$
这个公式类似交叉熵，衡量样本的纯度，为1或0标签的样本数目越多熵(entropy)越小
其中，$p_1$ 是标签为1在总样本的占比，$0 \text{log}_2(0) = 0$

In [37]:
def entropy(y):
    """
    Compute the entropy of a label array.
    
    Parameters:
    y (numpy array): Array of labels (0s and 1s).
    
    Returns:
    float: Entropy value.
    """
    if (len(y) == 0):
        return 0
    p1 = np.sum(y)/len(y)
    p0 = 1 - p1
    if p1 == 0 or p1 == 1:
        return 0
    return -p1 * np.log2(p1) - p0 * np.log2(p0)

In [38]:
# Sampling with replacement
def sampling(X_train:np.ndarray,y_train:np.ndarray)->tuple[np.ndarray,np.ndarray]:
    n = X_train.shape[0]
    indices = np.random.choice(n, size=n, replace=True)
    """
    sampling between 0 to n-1, size=n, with replacement(放回抽样)
    
    """
    return X_train[indices],y_train[indices]

In [39]:
# Split the node based on the feature seleted
def split_node(X:np.ndarray,y:np.ndarray,feature_index:int)->tuple[np.ndarray,np.ndarray,np.ndarray,np.ndarray]:
    """
    Split the dataset based on a feature index.
    
    Parameters:
    X (numpy array): Feature dataset.
    y (numpy array): Labels.
    feature_index (int): Index of the feature to split on.
    
    Returns:
    tuple: Four numpy arrays - X_left, y_left, X_right, y_right
    """
    X_left = X[X[:, feature_index] == 0]
    y_left = y[X[:, feature_index] == 0]
    X_right = X[X[:, feature_index] == 1]
    y_right = y[X[:, feature_index] == 1]
    
    return X_left, y_left, X_right, y_right

## Information Gain(== reduction of the entropy)


$$\text{Information Gain} = H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$

where 
- $H(p_1^\text{node})$ is entropy at the node 
- $H(p_1^\text{left})$ and $H(p_1^\text{right})$ are the entropies at the left and the right branches resulting from the split
- $w^{\text{left}}$ and $w^{\text{right}}$ are the proportion of examples at the left and right branch, respectively

In [40]:
# calculate information gain
def information_gain(y_father:np.array,y_left:np.ndarray,y_right:np.ndarray)->float:
    father_size = len(y_father)
    w_left = len(y_left)/father_size
    w_right = len(y_right)/father_size
    return entropy(y_father) - (w_left * entropy(y_left) + w_right * entropy(y_right))

In [41]:
# recursively split the dataset
def recursive_split(X:np.ndarray,y:np.ndarray,depth:int,max_depth:int)->dict:
    n_samples, n_features = X.shape
    # If all labels are the same, return a leaf node
    if len(set(y)) == 1: # len(y)?? 返回唯一标签的数量，set是python集合，可以去重，如果只有一种，则只有一个元素
        return {'type': 'leaf', 'class': y[0]}
    
    # If maximum depth is reached or no samples left, return a leaf node with majority class
    if depth == max_depth or n_samples == 0:
        majority_class = np.bincount(y).argmax()
        return {'type': 'leaf', 'class': majority_class}
    
    # Find the best feature to split on
    best_gain = -1
    best_feature = None
    best_splits = None
    
    # 这里允许了特征的重复使用！！！
    for feature_index in range(n_features):
        X_left, y_left, X_right, y_right = split_node(X, y, feature_index)
        gain = information_gain(y, y_left, y_right)
        
        if gain > best_gain:
            best_gain = gain
            best_feature = feature_index
            best_splits = (X_left, y_left, X_right, y_right)
    
    # If no information gain, return a leaf node with majority class
    if best_gain == 0:
        majority_class = np.bincount(y).argmax()
        return {'type': 'leaf', 'class': majority_class}
    
    # Recursively split the left and right nodes
    X_left, y_left, X_right, y_right = best_splits
    left_subtree = recursive_split(X_left, y_left, depth + 1, max_depth)
    right_subtree = recursive_split(X_right, y_right, depth + 1, max_depth)
    
    return {
        'type': 'node',
        'feature_index': best_feature,
        'left': left_subtree,
        'right': right_subtree
    }

In [42]:
def recursive_split_noReuse(X:np.ndarray, y:np.ndarray, depth:int, max_depth:int, used_features:set=None)->dict:
    n_samples, n_features = X.shape
    if used_features is None:
        used_features = set()
    
    # 如果所有标签相同，返回叶节点
    if len(set(y)) == 1:
        return {'type': 'leaf', 'class': y[0]}
    
    # 如果达到最大深度或没有样本，返回多数类叶节点
    if depth == max_depth or n_samples == 0:
        majority_class = np.bincount(y).argmax()
        return {'type': 'leaf', 'class': majority_class}
    
    best_gain = -1
    best_feature = None
    best_splits = None
    
    # 只考虑未使用过的特征
    available_features = [i for i in range(n_features) if i not in used_features]
    
    for feature_index in available_features:
        X_left, y_left, X_right, y_right = split_node(X, y, feature_index)
        gain = information_gain(y, y_left, y_right)
        
        if gain > best_gain:
            best_gain = gain
            best_feature = feature_index
            best_splits = (X_left, y_left, X_right, y_right)
    
    # 如果没有信息增益或没有可用特征，返回叶节点
    if best_gain == 0 or not available_features:
        majority_class = np.bincount(y).argmax()
        return {'type': 'leaf', 'class': majority_class}
    
    # 递归分裂，将当前使用的特征添加到已使用集合中
    X_left, y_left, X_right, y_right = best_splits
    new_used_features = used_features | {best_feature}
    
    left_subtree = recursive_split_noReuse(X_left, y_left, depth + 1, max_depth, new_used_features)
    right_subtree = recursive_split_noReuse(X_right, y_right, depth + 1, max_depth, new_used_features)
    
    return {
        'type': 'node',
        'feature_index': best_feature,
        'left': left_subtree,
        'right': right_subtree
    }

In [43]:
# random forest classifier
class RandomForestClassifier:
    def __init__(self, n_trees:int=10, max_depth:int=5):
        self.n_trees = n_trees
        self.max_depth = max_depth
        self.trees = []
    
    def fit(self, X:np.ndarray, y:np.ndarray):
        self.trees = []
        for _ in range(self.n_trees):
            X_sample, y_sample = sampling(X, y)
            tree = recursive_split(X_sample, y_sample, depth=0, max_depth=self.max_depth)
            self.trees.append(tree)
    
    def predict_single(self, x:np.ndarray, tree:dict)->int: # 其中x是单个样本，tree是决策树
        if tree['type'] == 'leaf':
            return tree['class']
        feature_value = x[tree['feature_index']]
        if feature_value == 0:
            return self.predict_single(x, tree['left'])
        else:
            return self.predict_single(x, tree['right'])
    def predict(self, X:np.ndarray)->np.ndarray:
        predictions = np.array([self.predict_single(x, tree) for x in X for tree in self.trees])# 列表初始化
        predictions = predictions.reshape(X.shape[0], self.n_trees)
        final_predictions = np.array([np.bincount(row).argmax() for row in predictions])
        return final_predictions

In [44]:
# try it out
rf = RandomForestClassifier(20,4)
rf.fit(X_train,y_train)
rf.trees


[{'type': 'node',
  'feature_index': 2,
  'left': {'type': 'node',
   'feature_index': 1,
   'left': {'type': 'leaf', 'class': np.int64(0)},
   'right': {'type': 'leaf', 'class': np.int64(1)}},
  'right': {'type': 'node',
   'feature_index': 0,
   'left': {'type': 'leaf', 'class': np.int64(0)},
   'right': {'type': 'leaf', 'class': np.int64(1)}}},
 {'type': 'node',
  'feature_index': 2,
  'left': {'type': 'node',
   'feature_index': 0,
   'left': {'type': 'leaf', 'class': np.int64(1)},
   'right': {'type': 'leaf', 'class': np.int64(0)}},
  'right': {'type': 'node',
   'feature_index': 0,
   'left': {'type': 'leaf', 'class': np.int64(0)},
   'right': {'type': 'leaf', 'class': np.int64(1)}}},
 {'type': 'node',
  'feature_index': 0,
  'left': {'type': 'leaf', 'class': np.int64(0)},
  'right': {'type': 'node',
   'feature_index': 2,
   'left': {'type': 'leaf', 'class': np.int64(0)},
   'right': {'type': 'leaf', 'class': np.int64(1)}}},
 {'type': 'node',
  'feature_index': 1,
  'left': {'ty

In [47]:
## 这里没有用到交叉验证
print("result:",rf.predict(X_train))
print("actual:",y_train)

result: [1 1 0 0 1 0 0 1 1 0]
actual: [1 1 0 0 1 0 0 1 1 0]
